In [1]:
import sys

sys.path.append("..")

import tensorflow as tf

import numpy as np

from utils import tf_config

dataset="01"
memory_limit=1500
n_individuals=10
interval_eeg=10

tf_config.set_seed(seed=42)
tf_config.setup_tensorflow(device="GPU", memory_limit=memory_limit)

import GPyOpt

import argparse

from utils import preprocess_data, search_algorithms, train, losses_utils, metrics, eeg_utils

from models import fmri_ae, eeg_to_fmri, uniconv_fmri

from layers import locally_connected

import matplotlib.pyplot as plt

import gc

import os

from sklearn.model_selection import train_test_split, KFold

from pathlib import Path

import time

import pickle

2022-01-14 15:10:15.054005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-14 15:10:16.247927: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-14 15:10:16.248406: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-14 15:10:16.285805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-14 15:10:16.286387: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:0a:00.0 name: GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2022-01-14 15:10:16.286417: I tensorflow/stream_executor/platform/def

## Build model

In [7]:
from benchmarks import liu

import importlib
importlib.reload(liu)
interval_eeg=1

with tf.device('/CPU:0'):
    model = liu.Liu_et_al((64,interval_eeg),(64,64,30,interval_eeg))
    
    optimizer = tf.keras.optimizers.Adam(0.001)
    loss_fn = losses_utils.mae

    train_set, test_set = model.load_data(dataset, n_individuals, time_length=interval_eeg, batch_size=4)

## Train

In [10]:
loss_history = train.train(train_set, model, optimizer, loss_fn, epochs=10, u_architecture=False, verbose=True)[0]

2022-01-14 15:28:48.203991: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 480.00MiB (rounded to 503316480)requested by op Conv2DBackpropFilter
Current allocation summary follows.
2022-01-14 15:28:48.204037: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-01-14 15:28:48.204050: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 1110, Chunks in use: 1110. 277.5KiB allocated for chunks. 277.5KiB in use in bin. 6.1KiB client-requested in use in bin.
2022-01-14 15:28:48.204060: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-01-14 15:28:48.204070: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 5, Chunks in use: 5. 5.2KiB allocated for chunks. 5.2KiB in use in bin. 5.0KiB client-requ

ResourceExhaustedError: OOM when allocating tensor with shape[1,1,1024,122880] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2DBackpropFilter]

## Metrics

In [9]:
importlib.reload(metrics)
rmse_pop = metrics.rmse(test_set, model)
ssim_pop = metrics.ssim(test_set, model)
print("RMSE: ", np.mean(rmse_pop), "\pm", np.std(rmse_pop))
print("SSIM: ", np.mean(ssim_pop), "\pm", np.std(ssim_pop))

2022-01-14 15:23:35.495077: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2022-01-14 15:23:36.311230: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2022-01-14 15:23:36.342561: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


RMSE:  0.9832776 \pm 0.12874115
SSIM:  0.001194277034655193 \pm 0.00028677889214190886
